# Import Tweets

In [35]:
# Import all libraries 
import pandas as pd
import numpy as np 
import datetime

In [36]:
trump_tweets = pd.read_csv("trump_tweets_cleaned.csv")
trump_tweets['date'] = pd.to_datetime(trump_tweets['date'])
trump_tweets.rename(columns={"date": "tweet_datetime"}, inplace=True)

In [37]:
trump_tweets

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month
0,9.384230e+17,MAKE AMERICA GREAT AGAIN!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6
1,9.391890e+17,MAKE AMERICA GREAT AGAIN!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8
2,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11
3,9.253890e+17,Thank you @LuisRiveraMarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10
4,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8
...,...,...,...,...,...,...,...,...,...,...,...
5083,9.862190e+17,"Employment is up, Taxes are DOWN. Enjoy! I am ...",106514,19661,2018-04-17 12:24:00,2018-04-17 20:24:00,2018-04-17,20:24:00,20,2018,4
5084,9.791090e+17,I am pleased to announce that I intend to nomi...,66173,13399,2018-03-28 21:31:00,2018-03-29 05:31:00,2018-03-29,05:31:00,5,2018,3
5085,9.597990e+17,Rasmussen just announced that my approval rati...,132165,30235,2018-03-02 14:40:00,2018-03-02 22:40:00,2018-03-02,22:40:00,22,2018,3
5086,9.576040e+17,Somebody please inform Jay-Z that because of m...,202727,49876,2018-01-28 13:18:00,2018-01-28 21:18:00,2018-01-28,21:18:00,21,2018,1


# Import Stock Prices

In [38]:
stock_price = pd.read_csv("stock_price_cleaned.csv")

# Get Stock Price Difference X mins after

In [39]:
# Change this variable for different X min differences
x_mins = 5

In [40]:
stock_price.rename(columns={"DATE_TIME_M": "datetime", "PRICE": "price"}, inplace=True)
stock_price['datetime']= pd.to_datetime(stock_price['datetime'])
stock_price[f"datetime_{x_mins}mins_before"] = stock_price["datetime"] - pd.Timedelta(f"{x_mins} min")

In [41]:
stock_price_merged = stock_price.merge(stock_price, how='inner', left_on=[f"datetime_{x_mins}mins_before"], right_on=["datetime"])
stock_price_merged.drop(columns=["datetime_y", f"datetime_{x_mins}mins_before_y"], inplace=True)
stock_price_merged.rename(columns={"datetime_x": f"datetime_{x_mins}mins_after", "price_x": f"price_{x_mins}mins_after", f"datetime_{x_mins}mins_before_x": "datetime_now", "price_y": "price_now"}, inplace=True)

In [42]:
stock_price

,datetime,price,datetime_5mins_before
0,2017-01-20 04:00:00,226.5000,2017-01-20 03:55:00
1,2017-01-20 04:01:00,226.4975,2017-01-20 03:56:00
2,2017-01-20 04:02:00,226.4950,2017-01-20 03:57:00
3,2017-01-20 04:03:00,226.4925,2017-01-20 03:58:00
4,2017-01-20 04:04:00,226.4900,2017-01-20 03:59:00
...,...,...,...
1023356,2018-12-31 19:56:00,250.0600,2018-12-31 19:51:00
1023357,2018-12-31 19:57:00,250.0700,2018-12-31 19:52:00
1023358,2018-12-31 19:58:00,250.1000,2018-12-31 19:53:00
1023359,2018-12-31 19:59:00,250.0700,2018-12-31 19:54:00


In [22]:
stock_price_merged['price_diff_abs'] = stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']

stock_price_merged['price_diff_perc'] = (stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

In [23]:
stock_price_merged

,datetime_5mins_after,price_5mins_after,datetime_now,price_now,price_diff
0,2017-01-20 04:05:00,226.520000,2017-01-20 04:00:00,226.500000,0.000088
1,2017-01-20 04:06:00,226.526667,2017-01-20 04:01:00,226.497500,0.000129
2,2017-01-20 04:07:00,226.533333,2017-01-20 04:02:00,226.495000,0.000169
3,2017-01-20 04:08:00,226.540000,2017-01-20 04:03:00,226.492500,0.000210
4,2017-01-20 04:09:00,226.530000,2017-01-20 04:04:00,226.490000,0.000177
...,...,...,...,...,...
1023351,2018-12-31 19:56:00,250.060000,2018-12-31 19:51:00,250.060000,0.000000
1023352,2018-12-31 19:57:00,250.070000,2018-12-31 19:52:00,250.090000,-0.000080
1023353,2018-12-31 19:58:00,250.100000,2018-12-31 19:53:00,250.100000,0.000000
1023354,2018-12-31 19:59:00,250.070000,2018-12-31 19:54:00,250.086667,-0.000067


# Merge Tweets and Stock Prices

In [24]:
merged = trump_tweets.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])
merged.drop(columns=["datetime_now", "price_now"], inplace=True)

In [25]:
merged.shape

(5087, 14)

In [26]:
merged

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month,datetime_5mins_after,price_5mins_after,price_diff
0,9.384230e+17,MAKE AMERICA GREAT AGAIN!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6,2017-06-12 15:05:00,242.920000,-0.000185
1,9.391890e+17,MAKE AMERICA GREAT AGAIN!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8,2017-08-12 17:51:00,244.775500,0.000010
2,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,2017-11-28 02:55:00,260.333750,0.000005
3,9.253890e+17,Thank you @LuisRiveraMarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10,2017-10-31 15:53:00,257.045000,-0.000467
4,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2017-08-22 01:05:00,243.287604,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,9.862190e+17,"Employment is up, Taxes are DOWN. Enjoy! I am ...",106514,19661,2018-04-17 12:24:00,2018-04-17 20:24:00,2018-04-17,20:24:00,20,2018,4,2018-04-17 12:29:00,270.210000,-0.000148
5083,9.791090e+17,I am pleased to announce that I intend to nomi...,66173,13399,2018-03-28 21:31:00,2018-03-29 05:31:00,2018-03-29,05:31:00,5,2018,3,2018-03-28 21:36:00,259.990000,0.000032
5084,9.597990e+17,Rasmussen just announced that my approval rati...,132165,30235,2018-03-02 14:40:00,2018-03-02 22:40:00,2018-03-02,22:40:00,22,2018,3,2018-03-02 14:45:00,268.040000,0.000000
5085,9.576040e+17,Somebody please inform Jay-Z that because of m...,202727,49876,2018-01-28 13:18:00,2018-01-28 21:18:00,2018-01-28,21:18:00,21,2018,1,2018-01-28 13:23:00,286.424812,-0.000001
